# End of week 1 solution

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

After week 2 you'll be able to add a User Interface to this tool, giving you a valuable application.

In [1]:
# imports

from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [3]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [4]:
# set up environment

load_dotenv()
openai = OpenAI()


In [5]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [6]:
# prompts

system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: " + question

In [7]:
# messages

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [8]:
# Get gpt-4o-mini to answer, with streaming

stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

To understand the code snippet `yield from {book.get("author") for book in books if book.get("author")}`, let's break it down step by step.

### Step 1: Breakdown of the Components

1. **Set Comprehension**:
   - `{book.get("author") for book in books if book.get("author")}` is a set comprehension.
   - It iterates over an iterable called `books`, which is presumably a collection (like a list or similar) of dictionaries.
   - For each `book` in `books`, it attempts to retrieve the value associated with the key `"author"` using `book.get("author")`.
   - The `if book.get("author")` part acts as a conditional filter, ensuring that only books with a non-`None` or non-falsy `author` field are included in the resulting set.

2. **Set Creation**:
   - The result of the set comprehension is a set of authors. Sets in Python are collections that contain only unique elements, which means if any author appears in multiple books, they will only appear once in the resulting set.

3. **Yield Statement**:
   - `yield from` is a special syntax in Python used with generators. It allows a generator function to yield all values from another iterable (in this case, from the set created).
   - When a generator function uses `yield from`, it effectively delegates part of its operations to another iterable (the authors' set, in this case), allowing it to yield multiple values from that iterable.

### Step 2: What the Code Does

Putting it all together, the line of code processes a collection of book dictionaries (`books`) to extract unique authors and yield each author from a generator function. Here is a more structured breakdown of its functionality:

- It collects all authors from the `books` collection.
- If a book has an associated author (i.e., the value is not `None` or any falsy value), that author is included in the set.
- It eliminates duplicates because a set can only contain unique values.
- Using `yield from`, it yields each unique author one at a time to whoever consumes the generator, making it an efficient way to produce results on-the-fly instead of returning a complete list all at once.

### Step 3: Usage Context

In practical terms, this piece of code might appear within a generator function, which could be defined as follows:

python
def unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}


For example, if you had a list of books like this:

python
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C", "author": None},
    {"title": "Book D", "author": "Author 1"},
]


Calling `list(unique_authors(books))` would return:


['Author 1', 'Author 2']


Here, `Author 1` appears only once in the output despite being present in multiple books, illustrating the uniqueness feature of the set.

### Summary

In summary, the code snippet is a generator expression that efficiently yields unique authors from a list of book dictionaries, filtering out any None or falsy values, and guaranteed to only provide unique authors. It exemplifies the power of set comprehension combined with generator functions for concise and efficient data processing in Python.

In [11]:
# Get Llama 3.2 to answer

response = ollama.chat(model=MODEL_LLAMA, messages=messages)
print(response)
reply = response['message']['content']
display(Markdown(reply))

model='llama3.2' created_at='2025-04-19T05:07:34.1342102Z' done=True done_reason='stop' total_duration=15777164700 load_duration=26637300 prompt_eval_count=85 prompt_eval_duration=9702500 eval_count=512 eval_duration=15738947900 message=Message(role='assistant', content='**Code Explanation**\n\nThe given code snippet is written in Python and utilizes several advanced features, including:\n\n1.  **Dictionary iteration**: The `in` keyword can be used to check if a key exists in a dictionary.\n2.  **List comprehension**: A concise way of creating lists.\n3.  **Generator expressions**: A compact way of creating generators.\n\nHere\'s the code snippet with some additional comments for better understanding:\n\n```python\n# Create a generator expression that filters books based on their \'author\'\nfiltered_authors = (\n    # Use list comprehension to get the author from each book\n    book.get("author")\n    # Iterate over all books in the \'books\' collection\n    for book in books\n    # C

**Code Explanation**

The given code snippet is written in Python and utilizes several advanced features, including:

1.  **Dictionary iteration**: The `in` keyword can be used to check if a key exists in a dictionary.
2.  **List comprehension**: A concise way of creating lists.
3.  **Generator expressions**: A compact way of creating generators.

Here's the code snippet with some additional comments for better understanding:

```python
# Create a generator expression that filters books based on their 'author'
filtered_authors = (
    # Use list comprehension to get the author from each book
    book.get("author")
    # Iterate over all books in the 'books' collection
    for book in books
    # Check if the 'author' key exists in each book
    if book.get("author")
)
```

However, there's a more concise way to write this using `yield from`:

```python
# Use yield from to filter books based on their 'author'
yield from (
    book.get("author")
    # Iterate over all books in the 'books' collection
    for book in books
    # Check if the 'author' key exists in each book
    if book.get("author")
)
```

**What does it do?**

This code creates a generator expression that yields the authors of books where the `author` key exists in the dictionary representation of each book.

Here's how you can use this generator expression:

```python
# Example usage:
books = [
    {"title": "Book 1", "author": "John"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Jane"}
]

for author in yield from (book.get("author") for book in books if book.get("author")):
    print(author)
```

**Why is it used?**

The `yield from` syntax is useful when you want to create a generator that composes multiple generators. In this case, we're creating a generator that filters the authors of books and then yielding each author individually.

Using `yield from`, we avoid having to manually iterate over the composed generator and instead let Python handle it for us.

This approach provides several benefits:

*   It's more concise than using an explicit loop.
*   It can improve performance by avoiding unnecessary intermediate collections.
*   It makes the code easier to read and understand, as the intent of the `yield from` statement is clear.

# Congratulations!

You could make it better by taking in the question using  
`my_question = input("Please enter your question:")`

And then creating the prompts and making the calls interactively.